In [1]:
%cd drive/MyDrive/collab_sandbox/NER/ner_tenses_recognition/
!ls

/content/drive/MyDrive/collab_sandbox/NER/ner_tenses_recognition
 1000_5_1_20_meaningful_classification.csv
 active_learning
 advanced_tutorial_crf_lstm.ipynb
 bestsellers.csv
 best-val-model-86.pt
 datasets
 index.html
 logs
 meaningful_classif
 meaningful_classif.ipynb
 med_5_2_21_medium_sentences_dataset.csv
 med_5_2_21_medium_sentences_dataset_labeled.csv
'med_5_2_21_medium_sentences_dataset_labeled__Sun May  2 19:36:07 2021_1619984167.8032556.csv'
 med_gen_4_23_21_medium_sentences_dataset.csv
 med_gen_4_23_21_medium_sentences_dataset_labeled.csv
 medium_articles
 medium_sentences_dataset-2-22-21.csv
 medium_sentences_dataset-2-22-21.gsheet
 medium_sentences_dataset_labeled.csv
 medium_sentences_dataset_labeled.gsheet
 medium_sentenses_labeled_all__4_23_21_80479.csv
'medium_sentenses_labeled_all__4_23_21_80479__Sun May  2 01:18:31 2021_1619918311.231216.csv'
 medium_sentenses_labeled_all__4_23_21_80511.csv
 medium_sentenses_labeled_all__5_2_21_88887.csv
 my_nerda
 NER_A1_tenses.ipy

In [2]:
!pip -q install transformers
import torch
import sklearn
import warnings
import transformers
# import sklearn.preprocessing
import numpy as np
from sklearn import preprocessing
from transformers import AdamW, get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup, get_cosine_with_hard_restarts_schedule_with_warmup
import random
from tqdm.notebook import tqdm
import torch.nn as nn
from transformers import AutoConfig

import pandas as pd
import os
import sys
from transformers import AutoModel, AutoTokenizer, AutoConfig
from typing import List
from typing import List
from sklearn.metrics import precision_recall_fscore_support
import warnings
from tqdm import tqdm_notebook

from tqdm.notebook import tqdm
import multiprocessing
from joblib import Parallel, delayed
num_cores = multiprocessing.cpu_count()
from concurrent.futures import ProcessPoolExecutor
from multiprocessing import freeze_support
from torch.utils.data import Dataset

import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, random_split, Dataset
import numpy as np
import spacy
from spacy import displacy

# active learning
from ipywidgets import GridspecLayout
from IPython.display import clear_output
import ipywidgets as widgets
from IPython.display import display
import pandas as pd
import os
import time
clear_output()

In [3]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(42)

In [ ]:
# def load_meaningful_sentences():
#   meaningful_sentences_1 = pd.read_csv("./meaningful_classif/meaningful_sentences_classification__1001__4_25_21__5_34.csv")
#   meaningful_sentences_1 = meaningful_sentences_1[(meaningful_sentences_1['Tag'] == 'yes') | (meaningful_sentences_1['Tag'] == 'no')]

#   meaningful_sentences_2 = pd.read_csv("./meaningful_classif/meaningful_sentences_classification_2_801__4_27_21.csv")
#   meaningful_sentences_2 = meaningful_sentences_2[(meaningful_sentences_2['Tag'] == 'yes') | (meaningful_sentences_2['Tag'] == 'no')]

#   meaningful_sentences = pd.concat([meaningful_sentences_1, meaningful_sentences_2], ignore_index=True)

#   return meaningful_sentences
  
# meaningful_sentences = load_meaningful_sentences()

# meaningful_sentences['Tag'].value_counts()

yes    1054
no      744
Name: Tag, dtype: int64

# Model

In [4]:
class MeaningfullDataset(Dataset):
    def __init__(self, dataset=None, tokenizer=None, inference=False):
      self.tokenizer = tokenizer 
      self.dataset = dataset.reset_index()
      self.labels = {'yes': 1, 'no': 0}
      self.inference = inference

    def __len__(self):
      return len(self.dataset)

    def __getitem__(self, idx):
        if not self.inference:
          label = self.dataset['Tag'][idx]
          label = self.labels[label]
          sentence = self.dataset['Text'][idx]
          
          encodings_dict = self.tokenizer(sentence,                                   
                                      truncation=True, 
                                      max_length=128, 
                                      padding="max_length")   
          
          input_ids = encodings_dict['input_ids']
          attention_mask = encodings_dict['attention_mask']
          
          return {'label': torch.tensor(label, device='cuda', dtype=torch.float),
                  'input_ids': torch.tensor(input_ids, device='cuda'), 
                  'attention_mask': torch.tensor(attention_mask, device='cuda')}
        else:
          sentence = self.dataset['Text'][idx]
          encodings_dict = self.tokenizer(sentence,                                   
                                      truncation=True, 
                                      max_length=128, 
                                      padding="max_length")   
          
          input_ids = encodings_dict['input_ids']
          attention_mask = encodings_dict['attention_mask']
          
          return {'input_ids': torch.tensor(input_ids, device='cuda'), 
                  'attention_mask': torch.tensor(attention_mask, device='cuda')}

from sklearn.metrics import accuracy_score as sklearn_accuracy

class ClassifModel(nn.Module):
    def __init__(self, transformer, n_tags=1, dropout = 0.1):
      super(ClassifModel, self).__init__()
      transformer_name = transformer.name_or_path
      transformer_config = AutoConfig.from_pretrained(transformer_name)
      self.transformer = transformer
      self.dropout = nn.Dropout(dropout)
      dimension  = transformer_config.hidden_size * 2
      self.output_1 = nn.Linear(dimension, 1)
      
      self.batch_norm_1 = torch.nn.BatchNorm1d(dimension)

      for parameter in self.transformer.parameters():
          parameter.requires_grad = False

      # print(self.transformer)
      for i, m in enumerate(self.transformer.encoder.layer):
          if i+1 > 12 - 1:
              for parameter in m.parameters():
                  parameter.requires_grad = True 
       

    def forward(self, 
                input_ids, 
                attention_mask,  
                ):
        transformer_inputs = {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            }
        outputs = self.transformer(**transformer_inputs, output_hidden_states=True)#[0][:, -1]
        layer_1_cls = outputs[2][-1][:, 0]
        layer_1_sep = outputs[2][-1][:, -1]
        outputs_1 = self.batch_norm_1(torch.cat([layer_1_cls, layer_1_sep, ], axis=-1))
        
        outputs_1 = self.dropout(outputs_1)
      
        outputs_1 = self.output_1(outputs_1)
        outputs_1 = outputs_1.squeeze(1)
        return torch.sigmoid(outputs_1)

class ClassifExperiment:
    def __init__(self, 
                 model_name = "roberta-base", 
                 dataset = None,
                 max_len = 128,
                 network = ClassifModel,
                 train_batch_size = 4,
                 dropout = 0.1,
                 grad_clip = 1,
                 optimizer_class = AdamW,
                 classes = 1,
                 target_field = 'Tag',
                 feature_field = 'Text'
                 ):
        self.device = 'cuda' 

        self.model_name = model_name  
        self.dataset = dataset
        self.max_len = max_len
        self.transformer_model = AutoModel.from_pretrained(model_name)
        self.transformer_tokenizer = AutoTokenizer.from_pretrained(model_name, 
                                                                   do_lower_case=False
                                                                   )
        self.transformer_config = AutoConfig.from_pretrained(model_name)  
        self.network = network(self.transformer_model, classes, dropout = dropout)
        self.network.to(self.device)
        self.train_losses = []
        self.valid_loss = np.nan
        self.train_batch_size = train_batch_size
        
        self.dl_train = None
        self.dl_validate = None
        self.dataset_validation = None
        self.target_field = target_field
        self.feature_field = feature_field
        self.train_examples = 100
        self.best_valid_loss = 10
        self.classes = classes

        self.scheduler = None
        self.scheduler_class = get_linear_schedule_with_warmup
        self.optimizer = None
        self.criterion = torch.nn.MSELoss()#torch.nn.BCELoss()#torch.nn.MSELoss()
        self.clip = grad_clip
        self.optimizer_class = optimizer_class
        self.setup()
        

    def experiment(self, 
                    epochs = 1,
                    warmup_steps = 300,
                    learning_rate = 5e-5
                   ):
        optimizer_parameters = self.network.parameters()

        num_train_steps = int(self.train_examples / self.train_batch_size * epochs)
        
        self.optimizer = self.optimizer_class(optimizer_parameters, lr = learning_rate)
        self.scheduler = self.scheduler_class(
            self.optimizer, num_warmup_steps = warmup_steps, num_training_steps = num_train_steps
        )

        train_losses = []
        best_parameters = None

        for epoch in range(epochs):
            
            print('\n Epoch {:} / {:}'.format(epoch + 1, epochs))

            train_loss = self.train()
            train_losses.append(train_loss)
            valid_loss = self.validate()

            print(f"Train Loss = {train_loss} Valid Loss = {valid_loss}")

            if valid_loss < self.best_valid_loss:
                best_parameters = self.network.state_dict()            
                self.best_valid_loss = valid_loss
        print(f"Best val => {self.best_valid_loss}")
        if best_parameters != None:
          self.network.load_state_dict(best_parameters)
        

        return "Model trained successfully"
    def setup(self):
      dataset_training, dataset_validation = train_test_split(self.dataset, test_size=0.25, random_state=42) 
      # dataset_training = pd.concat([dataset_training, load_meaningful_sentences_new()], axis=0, ignore_index=True)

      self.dataset_validation = dataset_validation
      self.train_examples = len(dataset_training)
      self.dl_train = self.create_dataloader(dataset = dataset_training, drop_last=True)
      self.dl_validate = self.create_dataloader(dataset = dataset_validation, drop_last=False)

    def create_dataloader(self, dataset, inference=False, drop_last=False):

      data_reader = MeaningfullDataset(
          dataset = dataset, 
          tokenizer = self.transformer_tokenizer, 
          inference=inference
          )

      shuffle = not inference
      data_loader = torch.utils.data.DataLoader(
          data_reader, batch_size = self.train_batch_size, num_workers = 0, shuffle=False, drop_last=drop_last
      )
      return data_loader

    def train(self,):
      self.network.train()    
      final_loss = 0.0
      
      for dl in self.dl_train:

          self.optimizer.zero_grad()
          # print(dl)
          outputs = self.network(
              input_ids=dl['input_ids'], 
              attention_mask=dl['attention_mask'],
          )
          loss = self.compute_loss(outputs, 
                              dl.get('label'),
                              dl.get('attention_mask'), 
                              )
          # print(loss)
          loss.backward()
          torch.nn.utils.clip_grad_norm_(self.network.parameters(), self.clip)
          self.optimizer.step()
          self.scheduler.step()
          final_loss += loss.item()

      return final_loss / len(self.dl_train)
    def validate(self):
        self.network.eval()
        final_loss = 0.0

        for dl in self.dl_validate:
            outputs = self.network(
              input_ids=dl['input_ids'], 
              attention_mask=dl['attention_mask'],
            )
            # print(outputs.shape, dl['input_ids'].shape)
            loss = self.compute_loss(outputs, 
                                dl.get('label'),
                                dl.get('attention_mask'), 
                                )
            final_loss += loss.item()
        
        return final_loss / len(self.dl_validate)   
    def compute_loss(self, preds, target_tags, masks,):
        active_logits = preds.view(-1, self.classes).squeeze(1)
        # print(preds.shape)
        # print(active_logits.shape, target_tags.shape)
        loss = self.criterion(active_logits, target_tags)
        return loss
    def evaluate_performance(self, dataset=None):
        if dataset is None:
          dataset = self.dataset_validation
        y_true = dataset[self.target_field].tolist()
        y_pred = self.predict(dataset)
        y_pred_1 = []
        for item in y_pred:
          y_pred_1.extend(item)

        f1 = precision_recall_fscore_support(y_true = y_true,
                                             y_pred = y_pred_1,
                                             labels = ['yes', 'no'],
                                             average=None) #'macro'
        # print(f1)
        df = list(zip(['no', 'yes'], f1[2]))
        df = pd.DataFrame(df, columns = ['Level', 'F1-Score'])    

        f1_micro = precision_recall_fscore_support(y_true = y_true,
                                             y_pred = y_pred_1,
                                             labels = ['yes', 'no'],
                                             average='micro') #'micro'
        f1_micro = pd.DataFrame({'Level' : ['AVG_MICRO'], 'F1-Score': [f1_micro[2]]})
        # accuracy = pd.DataFrame({'Level': ['acc'], "Accuracy": [sklearn_accuracy(y_true, y_pred_1)]})
        df = df.append(f1_micro)
        # df = df.append(accuracy)
      
        return df 
    def predict(self, dataset, pure_prob=False):
      self.network.eval()
      dl_test = self.create_dataloader(dataset=dataset, inference=pure_prob)

      predictions = []
      labels = ['no', 'yes']
      amount = 0
      with torch.no_grad():
        for i, dl in enumerate(dl_test): 
          outputs = self.network(
              input_ids=dl['input_ids'], 
              attention_mask=dl['attention_mask'],
          )   
          # print(outputs)
          if pure_prob:
            preds = outputs.cpu().numpy().tolist()
            predictions.extend(preds)
          else:
            preds = outputs.round().cpu().numpy().tolist()
            preds = [labels[int(item)] for item in preds]
            predictions.append(preds)
      return predictions


In [5]:
def load_meaningful_sentences_all():
  datasets_path = './active_learning/datasets/'
  concatenated_dataset = pd.DataFrame(columns=['Text'])
  datasets = os.listdir(datasets_path)

  for path in datasets:
    data = pd.read_csv(datasets_path+path)
    concatenated_dataset = pd.concat([concatenated_dataset, data], axis=0, ignore_index=True)

  concatenated_dataset = concatenated_dataset[concatenated_dataset['Text'].duplicated() == False]
  concatenated_dataset = concatenated_dataset[(concatenated_dataset['Tag'] == 'yes') | (concatenated_dataset['Tag'] == 'no')]
  return concatenated_dataset

def load_meaningful_sentences_new():
  datasets_path = './active_learning/datasets/'
  # concatenated_dataset = pd.DataFrame(columns=['Text'])
  # datasets = os.listdir(datasets_path)

  # for path in datasets:
  #   if 'mean_sents_exp_2' in path:
  #     data = pd.read_csv(datasets_path+path)
  #     concatenated_dataset = pd.concat([concatenated_dataset, data], axis=0, ignore_index=True)

  # concatenated_dataset = concatenated_dataset[concatenated_dataset['Text'].duplicated() == False]
  # concatenated_dataset = concatenated_dataset[
  #                                             (concatenated_dataset['Tag'] == 'yes') 
  #                                             # ]
  #                                             | 
  #                                             (concatenated_dataset['Tag'] == 'no')]
  # concatenated_dataset = concatenated_dataset[concatenated_dataset['Text'].duplicated() == False]
  new_dataset = pd.read_csv("./active_learning/datasets/meaningful_sentences_classification_3.csv")
  new_dataset = new_dataset[new_dataset['Text'].duplicated() == False]
  new_dataset = new_dataset[(new_dataset['Tag'] == 'yes') | (new_dataset['Tag'] == 'no')]
  # return concatenated_dataset
  return new_dataset

def load_meaningful_sentences_old():
  meaningful_sentences_1 = pd.read_csv("./meaningful_classif/meaningful_sentences_classification__1001__4_25_21__5_34.csv")
  meaningful_sentences_1 = meaningful_sentences_1[(meaningful_sentences_1['Tag'] == 'yes') | (meaningful_sentences_1['Tag'] == 'no')]

  meaningful_sentences_2 = pd.read_csv("./meaningful_classif/meaningful_sentences_classification_2_801__4_27_21.csv")
  meaningful_sentences_2 = meaningful_sentences_2[(meaningful_sentences_2['Tag'] == 'yes') | (meaningful_sentences_2['Tag'] == 'no')]

  meaningful_sentences = pd.concat([meaningful_sentences_1, meaningful_sentences_2], ignore_index=True)
  
  new_dataset = pd.read_csv("./active_learning/datasets/meaningful_sentences_classification_3.csv")
  new_dataset = new_dataset[new_dataset['Text'].duplicated() == False]
  new_dataset = new_dataset[(new_dataset['Tag'] == 'yes') | (new_dataset['Tag'] == 'no')]

  meaningful_sentences = pd.concat([meaningful_sentences, new_dataset], ignore_index=True)
  return meaningful_sentences

In [ ]:
my_classif.network

In [ ]:
meaningful_sentences = load_meaningful_sentences_all()

In [ ]:
meaningful_sentences[meaningful_sentences['Tag'] == 'no']

,Text,Tag,Markers
1,Those things are a part of yourself.,no,label_sentences__4_25_21_1000_
2,YPO has a lot of those.,no,label_sentences__4_25_21_1000_
4,Boo chases Mario if Mario is not facing Boo.,no,label_sentences__4_25_21_1000_
7,so we may better understand ourselves and how ...,no,label_sentences__4_25_21_1000_
8,But who on earth thought they should enter rig...,no,label_sentences__4_25_21_1000_
...,...,...,...
2986,it doesn't have to be like this.,no,1000_5_1_20_meaningful_classif
2987,If you know someone who has been abused and th...,no,1000_5_1_20_meaningful_classif
3000,and you are struggling to fix mistakes and fin...,no,1000_5_1_20_meaningful_classif
3011,The group's original tagline stated that it wa...,no,1000_5_1_20_meaningful_classif


In [6]:
meaningful_sentences = load_meaningful_sentences_all()
my_classif = ClassifExperiment(dataset=meaningful_sentences, dropout = 0.15,
                               model_name='roberta-base',
                               grad_clip = 0.5,
                               )

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
# my_classif.network.load_state_dict(torch.load("./meaningful_classif/meaningful_classif-0.75-0.187-4-29-21.pt")) - POTENTIAL DATA LEAK

<All keys matched successfully>

In [7]:
my_classif.network.load_state_dict(torch.load('./meaningful_classif/meaningful_classif-0.768__0.163_5_2_21.pt'))

<All keys matched successfully>

In [ ]:
len(load_meaningful_sentences_all())

3018

In [ ]:
my_classif.experiment(
    epochs = 4,
    warmup_steps = 200,
    learning_rate = 1e-5
)


 Epoch 1 / 4
Train Loss = 0.1508289139841444 Valid Loss = 0.16433624303333028

 Epoch 2 / 4
Train Loss = 0.14879083734961737 Valid Loss = 0.16360088063295555

 Epoch 3 / 4
Train Loss = 0.1486269612021876 Valid Loss = 0.16537575989902492

 Epoch 4 / 4
Train Loss = 0.14429499006096635 Valid Loss = 0.16535400118304308
Best val => 0.16360088063295555


'Model trained successfully'

In [ ]:
import gc
# del my_classif
gc.collect()
torch.cuda.empty_cache()
!nvidia-smi

In [ ]:
my_classif.evaluate_performance()

,Level,F1-Score
0,no,0.841342
1,yes,0.570025
0,AVG_MICRO,0.768212


In [ ]:
# torch.save(my_classif.network.state_dict(), './meaningful_classif/meaningful_classif-0.768__0.163_5_2_21.pt')

# labeler

In [ ]:
class ExampleSentence():
    def __init__(self, 
                 examples=None,
                 output_file=None,
                 dirname='./'
                 ):
      assert output_file != None, "output filename is None"
      # assert not output_file in os.listdir(f'{dirname}'), "File with this name already exists, be careful"
      
      self.output_file = output_file
      self.output_path = None
      self.dirname = dirname

      self.examples = examples
      self.current_elem_position = 0
      self.examples_len = len(examples) - 1
      self.current_value = "yes"
      self.current_example = examples[0]
      
      self.labeled_examples = examples
      self.labels = [self.current_value for _ in examples]
      ### elements
      self.progress = None
      self.toggle_buttons = None
      self.example_text = None
      self.next_button = None
      self.prev_button = None
      self.output = None
      self.grid = GridspecLayout(4, 4, width='800px', height='300px')
      ###
      self.add_elements()
      
      self.setup()
    def setup(self):
      self.toggle_buttons.observe(self.on_value_change, names='value',) 
      self.next_button.on_click(self.next_example)
      self.prev_button.on_click(self.prev_example)

      self.grid[:1, :] =  self.example_text
      self.grid[2, 1:3] = self.toggle_buttons
      self.grid[2, 3:] = self.next_button
      self.grid[2, :1] = self.prev_button
      self.grid[3, :2] = self.progress

      display(self.grid, self.output)

    def prev_example(self, some):
      if self.current_elem_position > 0:
        self.current_elem_position -= 1
        pos = self.current_elem_position
        self.progress.value = pos
        self.progress.description = f'{self.current_elem_position}/{len(self.examples)}'
        self.toggle_buttons.value = self.labels[pos]
        self.current_example = self.labeled_examples[pos]
        self.set_html_text(self.current_example)

    
    def next_example(self, some):
      if self.current_elem_position < self.examples_len:
        # self.labels[self.current_elem_position] = self.current_value
        # print(self.labels)
        self.current_elem_position += 1
        self.progress.value = self.current_elem_position
        self.progress.description = f'{self.current_elem_position}/{len(self.examples)}'
        self.current_example = self.examples[self.current_elem_position]
        self.toggle_buttons.value = self.labels[self.current_elem_position]
        self.set_html_text(self.current_example)

      elif self.current_elem_position == self.examples_len:
        # self.labels[self.current_elem_position - 1] = self.current_value
        # self.current_elem_position += 1
        self.progress.description = f'{self.current_elem_position+1}/{len(self.examples)}'
        print("End labeling")
        # TODO: some network train stuff
        # create dataframe etc
        self.create_dataframe()

    def create_dataframe(self):
        data = {
            'Text': self.examples,
            'Tag': self.labels
        }
        df = pd.DataFrame(data=data)
        timestamp = time.ctime()
        time_uniqueid = time.time()
        self.output_path = f"{self.dirname}{self.output_file}_{len(self.examples)}__{timestamp}_{time_uniqueid}.csv"
        df.to_csv(self.output_path, index=False)
        print(self.output_path)

    def on_value_change(self, change):
        with self.output:
          self.current_value = change['new']
          self.labels[self.current_elem_position] = self.current_value
          

    def set_html_text(self, text):
      render_html = f"""<p style="font-size: 1.3em;padding:16px 32px;">{text}</p>"""
      self.example_text.value = render_html

    def add_elements(self):
      self.progress = widgets.IntProgress(
              value=0,
              min=0,
              max=self.examples_len,
              description=f'{0}/{len(self.examples)}',
              bar_style='info',
              style={'bar_color': 'green'},
              orientation='horizontal'
          )
      self.toggle_buttons = widgets.ToggleButtons(
          options=['yes', 'no'],
          description='',
          disabled=False,
          button_style='',
      )
      
      self.example_text = widgets.HTML(value= "")
      self.set_html_text(self.current_example)

      self.next_button = widgets.Button(
          description='Next',
          disabled=False,
          button_style='',
          tooltip='Next',
          icon='check' 
      )
      self.prev_button = widgets.Button(
          description='Prev',
          disabled=False,
          button_style='',
          tooltip='Prev',
          icon='check' 
      )
      self.output = widgets.Output()

dataset_name = "meaningful_dataset_exp_2_30__Sat May  1 15:44:06 2021_1619883846.5970786.csv"
sentences_for_labeling = pd.read_csv(f'./active_learning/unlabeled_datasets/{dataset_name}')['Text'].tolist()
tog = ExampleSentence(examples=sentences_for_labeling,
                      output_file="mean_sents_exp_2",
                      dirname='./active_learning/datasets/'
                      )

GridspecLayout(children=(HTML(value='<p style="font-size: 1.3em;padding:16px 32px;">but for those who have the…

Output()

# Extract uncertain examples

In [ ]:
ALL_SENTENCES = pd.read_csv("./med_5_2_21_medium_sentences_dataset_labeled.csv")
ALL_SENTENCES['Text'] = ALL_SENTENCES['sentence']

In [ ]:
ALL_SENTENCES

,sentence,topic,tenses,Text
0,Any more stimulus will lead to much more infla...,economy,a1_future_simple,Any more stimulus will lead to much more infla...
1,The Commerce Department just released their es...,economy,a1_past_simple_reg,The Commerce Department just released their es...
2,GDP increased by an annual rate of 6.4%.,economy,a1_past_simple_reg,GDP increased by an annual rate of 6.4%.
3,"That means since May 1, 2020, the economy has ...",economy,a1_present_simple_3d_pers,"That means since May 1, 2020, the economy has ..."
4,With even higher growth numbers in the coming ...,economy,a1_to_be_present_is_am_are,With even higher growth numbers in the coming ...
...,...,...,...,...
31492,It also leverages global talent in service of us,travel,a1_present_simple_3d_pers,It also leverages global talent in service of us
31493,all - look no further than the San Francisco B...,travel,a1_to_be_present_is_am_are,all - look no further than the San Francisco B...
31494,"It's not easy, but not impossible; in fact, no...",travel,a1_to_be_present_is_am_are a1_present_simple_r...,"It's not easy, but not impossible; in fact, no..."
31495,"And despite far-right and GOP backlashes, dive...",travel,a1_present_continuous_act_rn,"And despite far-right and GOP backlashes, dive..."


In [ ]:
class UncertainDatacet():
  def __init__(self,
               model = None,
               source_dataset = None,
               labels_amount = 10,
               labeled_dataset_folder = None,
               output_dataset_folder = None,
               output_name = 'meaningful_dataset_exp_2'
               ):
    assert labeled_dataset_folder != None
    assert output_dataset_folder != None
    self.model = model
    self.labels_amount = labels_amount 
    self.source_dataset = source_dataset
    self.dataset_for_analysis = None
    self.labeled_dataset_folder = labeled_dataset_folder
    self.output_dataset_folder = output_dataset_folder

    self.output_dataset = None
    self.output_name = output_name
    self.output_path = None

  def create_dataset_for_analysis(self):
    concatenated_dataset = pd.DataFrame(columns=['Text'])
    datasets = os.listdir(self.labeled_dataset_folder)

    for path in datasets:
      data = pd.read_csv(self.labeled_dataset_folder+path)
      concatenated_dataset = pd.concat([concatenated_dataset, data], axis=0, ignore_index=True)

    concatenated_dataset = concatenated_dataset[concatenated_dataset['Text'].duplicated() == False]

    
    sentences = []
    concatenated_dataset_sentences = concatenated_dataset['Text'].tolist()
    for i in range(len(self.source_dataset)):
      text = str(self.source_dataset['Text'][i])
      in_dataset = text in concatenated_dataset_sentences
      if not in_dataset:
        sentences.append(text)
    dataset_for_analysis = pd.DataFrame(data={'Text': sentences})
    self.dataset_for_analysis = dataset_for_analysis
  def select_unsertain_sentences(self):
    predictions = self.model.predict(self.dataset_for_analysis, pure_prob=True)
    # print(predictions)
    uncertain_indices = np.abs(np.array(predictions, dtype=float) - 0.5).argsort()[:self.labels_amount]
    # print(uncertain_indices)
    most_uncertain_samples = self.dataset_for_analysis.iloc[uncertain_indices]
    # print(most_uncertain_samples)
    self.output_dataset = most_uncertain_samples
    timestamp = time.ctime()
    time_uniqueid = time.time()
    output_path = f"{self.output_dataset_folder}{self.output_name}_{self.labels_amount}__{timestamp}_{time_uniqueid}.csv"
    most_uncertain_samples.to_csv(output_path, index=False)
    self.output_path = output_path
    print(output_path)

    

In [ ]:
uncertain_datacet = UncertainDatacet(source_dataset=ALL_SENTENCES,
                 labeled_dataset_folder='./active_learning/datasets/',
                 output_dataset_folder='./active_learning/unlabeled_datasets/',
                 labels_amount = 30,
                 model=my_classif
                 )
uncertain_datacet.create_dataset_for_analysis()

In [ ]:
# uncertain_datacet.select_unsertain_sentences()

In [ ]:
def create_dataset_for_random_labeling(source_dataset):
  labeled_dataset_folder = './active_learning/datasets/'
  concatenated_dataset = pd.DataFrame(columns=['Text'])
  datasets = os.listdir(labeled_dataset_folder)

  for path in datasets:
    data = pd.read_csv(labeled_dataset_folder+path)
    concatenated_dataset = pd.concat([concatenated_dataset, data], axis=0, ignore_index=True)

  concatenated_dataset = concatenated_dataset[concatenated_dataset['Text'].duplicated() == False]

  
  sentences = []
  concatenated_dataset_sentences = concatenated_dataset['Text'].tolist()
  for i in range(len(source_dataset)):
    text = str(source_dataset['Text'][i])
    in_dataset = text in concatenated_dataset_sentences
    if not in_dataset:
      sentences.append(text)
  dataset_for_analysis = pd.DataFrame(data={'Text': sentences})
  dataset_for_analysis = dataset_for_analysis[dataset_for_analysis['Text'].duplicated() == False].sample(n=1000, random_state=42, axis=0)
  dataset_for_analysis.to_csv("1000_5_1_20_meaningful_classification.csv", index=False)

In [ ]:
# create_dataset_for_random_labeling(ALL_SENTENCES)

In [ ]:
# uncertain_datacet.dataset_for_analysis.to_csv('./active_learning/unlabeled_datasets/test_dataset.csv', index=False)

In [8]:
# len(ALL_SENTENCES) - len(uncertain_datacet.dataset_for_analysis)
def predict_noise_dataset(input_dataset_path):
  dataset = pd.read_csv(input_dataset_path)
  dataset['Text'] = dataset['sentence']
  predictions = my_classif.predict(dataset, pure_prob=True)
  # test_dataset['prob'] = predictions
  int_predictions = np.array(np.array(predictions).round(), dtype=int) == 1
  # labels = ['no', 'yes']
  # int_predictions = [labels[i] for i in int_predictions]
  # test_dataset['type'] = int_predictions
  dataset = dataset.iloc[int_predictions]
  dataset_name = input_dataset_path[:input_dataset_path.index(".csv")]
  timestamp = time.ctime()
  time_uniqueid = time.time()
  output_path = f"{dataset_name}__{timestamp}_{time_uniqueid}.csv"

  dataset = dataset[['sentence', 'topic', 'tenses']]
  dataset.to_csv(output_path, index=False)
  print(f'Dataset len => {len(dataset)}')
  print(dataset.head(5))

In [9]:
# predict_noise_dataset("./med_5_2_21_medium_sentences_dataset_labeled.csv")
predict_noise_dataset("/content/drive/My Drive/collab_sandbox/NER/ner_tenses_recognition/datasets/comb_med_2119_6_15_21/comb_med_2119_6_17_21_sentences_labeled.csv")

Dataset len => 88696
                                            sentence  ...                                             tenses
0  It pollutes the environment, it's limited in s...  ...  a1_present_simple_3d_pers a1_to_be_present_is_...
1  Of course, nowadays electric cars are undoubte...  ...  a1_to_be_present_is_am_are a1_comparative_long...
2  Yet, it's worth talking about biofuels, as the...  ...  a1_to_be_present_is_am_are a1_present_simple_r...
3  While there are many types of biofuels, we'll ...  ...  a1_there_is_am_are a1_future_simple a1_superla...
4  There are also biofuels for planes and boats, ...  ...                                 a1_there_is_am_are

[5 rows x 3 columns]


In [ ]:
"/test_dataset.csv"[:"/test_dataset.csv".index(".csv")]

'/test_dataset'

In [ ]:
int_predictions = np.array(np.array([0.4, 0.51, 0.6, 0.1, 0.4, 0.2]).round(), dtype=int).tolist()

[0, 1, 1, 0, 0, 0]

In [ ]:
cleaned_dataset = pd.read_csv("./test_dataset_predictions.csv")

cleaned_dataset = cleaned_dataset[cleaned_dataset['type'] == 'yes']
cleaned_dataset = 